# 1. Importation des bibliothèques

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib
import os

# 2. Chargement des données

In [2]:
# Définir le chemin du fichier prétraité
file_path = "/Users/ines/Desktop/fetal_health_project/data/fetal_health_preprocessed.csv"

# Charger les données prétraitées
df = pd.read_csv(file_path)

# 3. Séparation des variables (features et cible)

In [3]:
# Séparer les features et la cible
X = df.drop(columns=["fetal_health"])  # Variables indépendantes
y = df["fetal_health"]  # Variable cible

# 4. Division en ensembles d'entraînement et de test

In [4]:
# Diviser les données en jeu d'entraînement (80%) et de test (20%)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 5. Entraînement du modèle

In [5]:
# Initialiser le modèle Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)

# Entraîner le modèle sur les données d'entraînement
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

# 6. Prédictions sur l'ensemble de test

In [6]:
# Faire des prédictions sur les données de test
y_pred = model.predict(X_test)

# 7. Évaluation des performances du modèle

In [7]:
# Calcul des métriques d'évaluation
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)

# 8. Sauvegarde du modèle

In [8]:
# Définir le chemin de sauvegarde du modèle
model_path = "/Users/ines/Desktop/fetal_health_project/data/fetal_health_model.pkl"

# Sauvegarder le modèle entraîné
joblib.dump(model, model_path)

['/Users/ines/Desktop/fetal_health_project/data/fetal_health_model.pkl']